<img src="media/docker_logo_1.png" width="500" height="500">

It is probably fair to say that access to servers has never been easier.  With platforms such as AWS, Azure, and Google GCE we can now launch on-demand servers of all varieties and configurations.  This programable infrastructure (IaaS) help companies, agencies, and institutions maintain agility as market and mission pressures evolve. However, even with the rise of IaaS, application packaging, configuration, and composition has not advanced despite considerable efforts in configuration management.  This is where [`docker`](https://www.docker.com/) comes in.  

`Docker` is not about full virtualization but rather about the ease of packaging and running applications using [Linux containers](https://en.wikipedia.org/wiki/LXC).  The idea is that `docker` containers wrap a piece of software or application in a complete filesystem that contains everything needed to run: code, runtime, system tools, system libraries (i.e. anything that can be installed on a server). This guarantees that the software will always run the same everywhere, regardless of the OS/compute environment running the container. Docker also provides portable Linux deployment such that containers can be run on any Linux system with kernel is 3.10 or later.  All major Linux distros have supported Docker since 2014.  While no doubt containers and virtual machines have similar resource isolation and allocation benefits, the architectural approach of Linux containers allows containerized applications to be more portable and efficient.

At NVIDIA, we use containers in a variety of ways including development, testing, benchmarking, and of course in production as the mechanism for deploying deep learning frameworks. Using [`nvidia-docker`](https://github.com/NVIDIA/nvidia-docker), a light-weight `docker` plugin, we can develop and prototype GPU applications on a workstation, and then deploy those applications anywhere that supports GPU containers.

# Setup
In the interest of time, we've already configured docker and nvidia-docker. If you're interested in setup details, see Appendix A at the bottom.

# First Contact

The simplest way to interact with `docker` is probably to just ask for the version information

In [11]:
docker --version

Docker version 1.12.3, build 6b644ec


We can ask `nvidia-docker` for the version information too

In [12]:
nvidia-docker --version

Docker version 1.12.3, build 6b644ec


Notice that `nvidia-docker` invocation here was simply "pass through" to `docker` command itself.

Next best way to get familiar with docker command line is to ask for `--help`

In [13]:
docker --help

Usage: docker [OPTIONS] COMMAND [arg...]
       docker [ --help | -v | --version ]

A self-sufficient runtime for containers.

Options:

  --config=~/.docker              Location of client config files
  -D, --debug                     Enable debug mode
  -H, --host=[]                   Daemon socket(s) to connect to
  -h, --help                      Print usage
  -l, --log-level=info            Set the logging level
  --tls                           Use TLS; implied by --tlsverify
  --tlscacert=~/.docker/ca.pem    Trust certs signed only by this CA
  --tlscert=~/.docker/cert.pem    Path to TLS certificate file
  --tlskey=~/.docker/key.pem      Path to TLS key file
  --tlsverify                     Use TLS and verify the remote
  -v, --version                   Print version information and quit

Commands:
    attach    Attach to a running container
    build     Build an image from a Dockerfile
    commit    Create a new image from a container's changes
    cp        Copy files/folde

The format of `docker` command line interactions is: 

`docker [OPTIONS] COMMAND [arg...]` 

and as the help display shows there are a lot of commands to choose from.  Don't worry, much like a big city, once we become more familiar with these commands the list won't seem so big.  We can start to drill down and get help that is specific to each command.  For example, one of the most useful `docker` commands is **`images`** which list all local containers on the host that `docker` knows about 

In [15]:
docker images --help


Usage:	docker images [OPTIONS] [REPOSITORY[:TAG]]

List images

Options:
  -a, --all             Show all images (default hides intermediate images)
      --digests         Show digests
  -f, --filter value    Filter output based on conditions provided (default [])
      --format string   Pretty-print images using a Go template
      --help            Print usage
      --no-trunc        Don't truncate output
  -q, --quiet           Only show numeric IDs


OK, lets now ask `docker` about the what container images are available locally on the host

In [10]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        5 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        5 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        5 months ago        1.848 kB


Here the output specifies three container images with some general metadata associated with each one.  First you'll notice that the images are quite large on average (~ 2GB) and that each image is associated with a unique ID hash.  When containers are created (i.e. via the **`create`** command) they are created from images.  There is no limit to the number of containers we can create from an images so it is important that `docker` associates UUIDs for each image and container.  Notice the `REPOSITORY` and `TAG` columns here specify more human readable image labels.  The repository loosely coresponds to the image name (i.e. url) and just as in the version control system [GIT](https://git-scm.com/) images can be modified and "tagged" rather than explicitly changing the image name for each image version. 

Here we have the "`nvidia/cuda`" container with ID `c54a2cc56cbb` and is tagged as the "`latest`" version of the image (i.e. most current).  The deep learning library [`cuDNN`](https://developer.nvidia.com/cudnn) was added to the image and a new image was created under the same name but tagged appropriately as "`8.0-cudnn5-devel`".

You're probably wondering already "*where does docker store these containers?*".  In general, docker works in `/var/lib/docker` and images are stored in `image` subdirectory.  For more information and details about where and how docker stores images on the host machine, see [here](https://stackoverflow.com/questions/19234831/where-are-docker-images-stored-on-the-host-machine#). 

For now just know that docker works with "images" and all containers are created from these images.  We will go into all the details about creating and modifying images etc in just a bit.  But first, lets actually kick around some containers!

## Getting Started with Containers

First things first lets have docker list *all* containers using the **`ps`** command

In [19]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


Again, don't forget you can get help for each command with `docker [COMMAND] --help`.  Use this to get additional details on the **`ps`** command.

Lets now use the docker command **`create`** to initialize a container from the `nvidia/cuda:latest` image

In [20]:
docker create nvidia/cuda:latest

9a12d94b12fd4e3de47a3ee3bab4d771d80b6895b575dc2c1ad257cb0f3d121f


The responce we recieved is a sha256 UUID for the generated container and listing `docker` containers again we see this new container now listed 

In [21]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS              PORTS               NAMES
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         45 seconds ago      Created                                 lonely_easley


It is important to understand that the container is not actually doing anything right now.  We've only "stamped" out a container from an image -- the container is not running at this point.  Were the container active the `STATUS` would read "running".  OK, so what is the container doing there?  Well the answer is "nothing".  Think about when we enter commands on the command-line -- each time we hit enter we implicitly specify that we would like that command to be executed immediately.  You can think of containers as a command that has not yet executed.  This command is wrapped up in the container and has all the resources (libraries etc) needed for successful execution.  Speaking of which, lets actually run this container ... 

In [22]:
nvidia-docker run 9a12d94b12fd

Using default tag: latest
Pulling repository docker.io/library/9a12d94b12fd
nvidia-docker | 2016/11/30 01:01:55 Error: image library/9a12d94b12fd:latest not found


hm ... using the **`run`** command seemed like a good guess.  Lets try the **`start`** command instead

In [23]:
nvidia-docker start 9a12d94b12fd

9a12d94b12fd


OK, that looks better. Using the **`start`** command docker returned the sha256 UUID.  Lets have a look at the docker containers again

In [27]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS                     PORTS               NAMES
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         About an hour ago   Exited (0) 6 minutes ago                       lonely_easley


Now the status says "Exited (0) ...".  Notice that the command (i.e. entry point) is `/bin/bash`.  When the **`start`** was issued the "COMMAND" was executed and by definition *bash command language interpreter that executes commands read from the standard input or from a file*.  However, there were no commands to execute from standard input!  Containers can have other entry points -- the reason `/bin/bash` is used most often is that it allows the container to act more generically as a shell so we can send it additional instructions. Note that all containers have a default entrypoint of "`/bin/sh -c`" unless otherwise specified.

Here our hands are tied with what the container will do.  Each time we issue the **`start`** command the container executes the entrypoint "`/bin/bash`" and since there is nothing on standard input the container simply exits.  This is where the **`run`** command comes in.

Instead of creating and starting a container explicitly we can use the **`run`** command to exectue a command within a particular image via creating a container from that image with the appropriate entrypoint.  Lets issue a **`run`** command passing the image ID of the "`nvidia/cuda:latest`" image as the argument.

In [31]:
nvidia-docker run 367795fb1051

Notice that the command **`start`** takes a container ID as the argument while the **`run`** command takes an image ID.  Lets have a look at the containers

In [33]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED              STATUS                          PORTS               NAMES
b79dee54ffe1        367795fb1051         "/bin/bash"         About a minute ago   Exited (0) About a minute ago                       amazing_euler
ff763f6099b1        367795fb1051         "/bin/bash"         3 minutes ago        Exited (0) 2 minutes ago                            suspicious_pike
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         2 hours ago          Exited (0) 18 minutes ago                           lonely_easley


Now we have an additional container both from image `367795fb1051` which have exited.  At this point the **`run`** command has done exactly what the **`start`** command has done (i.e. started a container which executed the entrypoint and exited).  However, the docker **`run`** command allows us to pass an alternative command to the container (`docker run --help`).  Lets try to pass an alternative instruction.

In [34]:
nvidia-docker run 367795fb1051 nvidia-smi

Wed Nov 30 01:49:50 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GRID K520           Off  | 0000:00:03.0     Off |                  N/A |
| N/A   26C    P8    17W / 125W |      0MiB /  4036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Finally!  Just to be clear, the `nvidia-smi` command was exectued within the container -- not the host.  Lets have a look at the containers yet again

In [36]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS                      PORTS               NAMES
2747af7b70a0        367795fb1051         "nvidia-smi"        19 seconds ago      Exited (0) 18 seconds ago                       berserk_goldwasser
b79dee54ffe1        367795fb1051         "/bin/bash"         7 minutes ago       Exited (0) 7 minutes ago                        amazing_euler
ff763f6099b1        367795fb1051         "/bin/bash"         9 minutes ago       Exited (0) 9 minutes ago                        suspicious_pike
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         2 hours ago         Exited (0) 24 minutes ago                       lonely_easley


We now have a new container from image `367795fb1051` but the "COMMAND" has been set to `nvidia-smi` as instructed by our **`run`** command.  Just for kicks lets issue a **`start`** command to this new container.  Each container gets a uuid that will change every time this lab is run so make sure to replace the container ID in the command below with the appropriate container ID listed above.

In [39]:
docker start 2747af7b70a0

2747af7b70a0


Now wait a minute, where is our output??

In [40]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS                      PORTS               NAMES
2747af7b70a0        367795fb1051         "nvidia-smi"        11 minutes ago      Exited (0) 3 seconds ago                        berserk_goldwasser
b79dee54ffe1        367795fb1051         "/bin/bash"         19 minutes ago      Exited (0) 19 minutes ago                       amazing_euler
ff763f6099b1        367795fb1051         "/bin/bash"         20 minutes ago      Exited (0) 20 minutes ago                       suspicious_pike
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         2 hours ago         Exited (0) 36 minutes ago                       lonely_easley


Sure enough when we check the docker container status it has status of "Exited (0) 10 seconds ago ..." which means that the **`start`** command did indeed start the container. The long story short is that the **`run`** command automatically provides the standard output from the command specified where as **`start`** does not forward the stdout by default -- we have to explicitly ask.  According to the help section for the **`start`** command, the option "`--attach`" attaches STDOUT/STDERR and forward signals. 

In [45]:
docker start --attach 2747af7b70a0

Wed Nov 30 02:10:54 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GRID K520           Off  | 0000:00:03.0     Off |                  N/A |
| N/A   26C    P8    17W / 125W |      0MiB /  4036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Bingo!

A few final words on starting and running containers. Keep an eye out on the container list when using the **`run`** command as each invocation creates a *new* image.  There is no problem having many (many) container stitting around but eventually it creates clutter.  Remember, containers are ment to be light-weight and disposable.  To that end lets clean up our containers.

In [51]:
# generate a list of container ID from the docker ps command
docker ps -a | awk '{print $1}' | tail -n +2

2747af7b70a0
b79dee54ffe1
ff763f6099b1
9a12d94b12fd


In [52]:
# for each container ID use the docker "rm" command to remove/delete the container
for cid in $(docker ps -a | awk '{print $1}' | tail -n +2);do docker rm $cid; done

2747af7b70a0
b79dee54ffe1
ff763f6099b1
9a12d94b12fd


In [53]:
# check docker containers one last time
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


All clean!

### Discovery Requires Experimentation
Make sure that you're comfortable creating containers and executing commands before moving forward.  Docker is quite forgiving so do be afraid to try lots of different things out while you explore.  A few ideas of things to try are 
1. Get the container to ping google
2. Maybe try `ifconfig` inside of the container
3. What does the container return when asked for disk usage (i.e `df -h`)??

In [55]:
nvidia-docker run 367795fb1051 ping -t1 www.google.com

PING www.google.com (172.217.5.4) 56(84) bytes of data.
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=1 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=2 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=3 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=4 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=5 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=6 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=7 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=8 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=9 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=10 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=11 Time to live exceeded
From ip-172-17-0-1.ec2.internal (172.17.0.1) icmp_seq=12 Time to live exceeded
From 

# Diving Deeper into Containers and Image Creation

In [3]:
nvidia-docker run --rm nvidia/cuda:8.0-cudnn5-devel nvidia-smi

Sat Nov 12 21:09:56 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [5]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [6]:
nvidia-docker run nvidia/cuda:8.0-cudnn5-devel nvidia-smi

Sat Nov 12 21:11:30 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [8]:
docker ps -a

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS                      PORTS               NAMES
e310b48e17f9        nvidia/cuda:8.0-cudnn5-devel   "nvidia-smi"        12 seconds ago      Exited (0) 11 seconds ago                       backstabbing_engelbart


In [9]:
nvidia-docker start e310b48e17f9

e310b48e17f9


In [10]:
nvidia-docker start --help


Usage:	docker start [OPTIONS] CONTAINER [CONTAINER...]

Start one or more stopped containers

Options:
  -a, --attach               Attach STDOUT/STDERR and forward signals
      --detach-keys string   Override the key sequence for detaching a container
      --help                 Print usage
  -i, --interactive          Attach container's STDIN


In [11]:
nvidia-docker start -i e310b48e17f9

Sat Nov 12 21:16:36 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
nvidia-docker start -a e310b48e17f9

Sat Nov 12 21:17:01 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [13]:
nvidia-docker exec e310b48e17f9 echo "hello from nvidia"

Error response from daemon: Container e310b48e17f97c1103c53a3c91d008581019782cb523462c68dbbcebc4043e54 is not running


In [14]:
nvidia-docker run --interactive --tty -d e310b48e17f9 /bin/bash

Using default tag: latest
Pulling repository docker.io/library/e310b48e17f9
nvidia-docker | 2016/11/12 21:29:35 Error: image library/e310b48e17f9:latest not found


In [16]:
docker ps -a

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS                      PORTS               NAMES
e310b48e17f9        nvidia/cuda:8.0-cudnn5-devel   "nvidia-smi"        20 minutes ago      Exited (0) 14 minutes ago                       backstabbing_engelbart


In [17]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        3 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        3 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        4 months ago        1.848 kB


In [18]:
nvidia-docker run --interactive --tty -d nvidia/cuda:8.0-cudnn5-devel /bin/bash

1868fbf74e0b9e613b4a41760f2b31257c280624a3b70d4705d4cb8c4f519150


In [19]:
docker ps 

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS              PORTS               NAMES
1868fbf74e0b        nvidia/cuda:8.0-cudnn5-devel   "/bin/bash"         16 seconds ago      Up 15 seconds                           stupefied_bhabha


In [26]:
nvidia-docker exec 1868fbf74e0b ls /usr/local/cuda/extras

CUPTI
Debugger


# Appendix A
Here are some installation details for getting docker up and running from scratch ...

### Step 0 - GPU Driver
In order to get GPU access with in docker/nvidia-docker we need to make sure that the NVIDIA driver is available on the host system.  It's possible to obtain the appropriate device driver from either the standard [driver download](http://www.nvidia.com/download/index.aspx) page or via [CUDA installation](https://developer.nvidia.com/cuda-downloads).

### Step 1 - Docker Install
Once the NVIDIA device driver has been successfully installed, we need to install `docker` it self. The installation of docker is quite simple but it is just slightly different for each OS.  The steps for `docker` installation on Ubuntu 14.04 can be found [here](https://docs.docker.com/engine/installation/linux/ubuntulinux/).  Don't worry, the [`docker` docs](https://docs.docker.com/) have install instructions for many other operating systems including RedHat, CentOS, Debian, and so on.

Docker provides an official installation script via https://get.docker.com which can accessed via command-line using "`wget -qO-`" or "`curl -sSL`"

### Step 2 - NVIDIA Docker 
The final configuration step is to obtain the `nvidia-docker` plugin which properly exposes the GPU hardware and drivers for `docker` containers.  Official installation instructions for `nvidia-docker` for Ubuntu, CentOS, and other distributions can be found [here](https://github.com/NVIDIA/nvidia-docker)